In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [26]:
train_json=r'E:\iiit\Sem3\Information Extraction And Retrieval\major_project\data\json\train_data.json'
val_json=r'E:\iiit\Sem3\Information Extraction And Retrieval\major_project\data\json\val_data.json'

In [33]:
df_train=pd.read_json(train_json,orient='table')

In [31]:
df_test=pd.read_json(val_json,orient='table')

In [34]:
df_train.hyperpartisan.value_counts()

true     300000
false    300000
Name: hyperpartisan, dtype: int64

In [32]:
df_test.hyperpartisan.value_counts()

false    75000
true     75000
Name: hyperpartisan, dtype: int64

Remove the tweets which have multiple emotions


In [35]:
labels=df_train.hyperpartisan.unique()
label_dict={}
for index,label in enumerate(labels):
  label_dict[label]=index

In [8]:
# label_dict.items()

In [6]:
df_train['label']=df_train.hyperpartisan.replace(label_dict)
df_train

,published-at,clean_text,clean_title,bias,hyperpartisan,labeled_by,label
1,2017-09-10,when explaining her decision to reevaluate tit...,after devos announced plans to reexamine title...,right,true,publisher,0
2,2017-05-05,a florida university will honor trayvon martin...,university to award trayvon martin with posthu...,right,true,publisher,0
8,2017-11-15,nov 15 upi texas state university has suspende...,texas state university suspends greek life aft...,right-center,false,publisher,1
12,2016-09-26,against the backdrop of an increasingly polari...,jewish organization's huge day of unity on tue...,right,true,publisher,0
15,2016-11-29,president elect donald trump has reached an ag...,"breaking: trump reaches agreement to keep 1,00...",right,true,publisher,0
...,...,...,...,...,...,...,...
1494880,2017-12-07,los angeles ap john boyega may have a lot in c...,"john boyega talks 'last jedi,' personal parall...",least,false,publisher,1
1494882,2011-12-09,kiribati in climate negotiations in durban sou...,sinking feeling: more bad news for pacific isl...,left,true,publisher,0
1494887,unknown,the little red hamburger hut near old town is ...,"like home: big, juicy burgers made the old-fas...",least,false,publisher,1
1494891,2017-11-12,it s an important week for the republican led ...,decision time for ny and nj republicans on tax...,left-center,false,publisher,1


We see that there is  huge imbalance of sentiment distribution,we need to take them into account.


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
x_train,x_val,y_train,y_val=train_test_split(df_train.index.values,df_train.label.values,test_size=0.1,stratify=df_train.label.values)

In [9]:
df_train.loc[x_train,'data_type']='train'
df_train.loc[x_val,'data_type']='val'

In [2]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [15]:
#add special tokens is to tell bert where the sentence ends and where it begins
#return attention mask is used for variable length sentences
#return tensors to specify tensor for tensorflow or pytorch

In [11]:
tokenizer=BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [12]:
encoded_data_train=tokenizer.batch_encode_plus(
    df_train.clean_text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=128,
    return_tensors='pt'
)


In [13]:
encoded_data_val=tokenizer.batch_encode_plus(
    df_train[df_train.data_type=='val'].clean_text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=128,
    return_tensors='pt'
)

In [14]:
encoded_data_test=tokenizer.batch_encode_plus(
    df_test.clean_text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=128,
    return_tensors='pt'
)

In [16]:
# encoded_data_train

what bert requires for training is below

In [17]:
# encoded_data_test

In [15]:
input_ids_train=encoded_data_train['input_ids']
attention_masks_train=encoded_data_train['attention_mask']
labels_train=torch.tensor(df_train.label.values)


input_ids_val=encoded_data_val['input_ids']
attention_masks_val=encoded_data_val['attention_mask']
labels_val=torch.tensor(df_train[df_train.data_type=='val'].label.values)



input_ids_test=encoded_data_test['input_ids']
attention_masks_test=encoded_data_test['attention_mask']
#labels_val=torch.tensor(df[df.data_type=='val'].label.values)

In [16]:
dataset_train=TensorDataset(input_ids_train,
                            attention_masks_train,labels_train)

dataset_val=TensorDataset(input_ids_val,
                            attention_masks_val,labels_val)

dataset_test=TensorDataset(input_ids_test,
                            attention_masks_test)

In [12]:
from transformers import BertForSequenceClassification

In [4]:
model=BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)

In [22]:
# model

CREATE TORCH DATALOADERS

In [2]:
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

In [3]:
bs=16

In [21]:
dataloader_train=DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=bs
)

dataloader_val=DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=bs
)

In [22]:
dataloader_test=DataLoader(
    dataset_test,
    shuffle=False,
    batch_size=bs
)

In [39]:
# torch.save(dataloader_train,'train_bert.pth')
# torch.save(dataloader_val,'val_bert.pth')
# torch.save(dataloader_test,'test_bert.pth')



torch.save(dataloader_train,'train_bert16.pth')
torch.save(dataloader_val,'val_bert16.pth')
torch.save(dataloader_test,'test_bert16.pth')

In [4]:
dataloader_train=torch.load('train_bert16.pth')
dataloader_test=torch.load('test_bert16.pth')
dataloader_val=torch.load('val_bert16.pth')

In [10]:
dataloader_train=torch.load('train_bert.pth')
dataloader_test=torch.load('test_bert.pth')
dataloader_val=torch.load('val_bert.pth')

SETUP/DEFINE THE OPTIMIZER

In [5]:
from transformers import AdamW,get_linear_schedule_with_warmup

In [9]:
optimizer=AdamW(
    model.parameters(),
    lr=1e-5,
    eps=1e-8
)

In [10]:
num_epochs=3
scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*num_epochs
)

DEFINE PERFORMANCE METRICS

In [6]:
import numpy as np
from sklearn.metrics import f1_score

In [18]:
#got preds in form [0.9 0.5 0.5 0 0]
#want preds in form[1 0 0 0 0 ]
def get_f1_score(predictions,labels):
  predictions_flat=np.argmax(predictions,axis=1).flatten()
  labels_flat=labels.flatten()
  return f1_score(labels_flat,predictions_flat,average='weighted')

In [19]:
def accuracy_per_class(predictions,labels):
  inverse_label_dict={value:key for key,value in label_dict.items()}

  predictions_flat=np.argmax(predictions,axis=1).flatten()
  labels_flat=labels.flatten()

  for label in np.unique(labels_flat):
    y_preds=predictions_flat[labels_flat==label]
    y_true=labels_flat[labels_flat==label]
    print(f'Classs : {inverse_label_dict[label]} : Accuracy {len(y_preds[y_preds==label])}/{len(y_true)}\n')
  

TRAINING

In [7]:
if torch.cuda.is_available():
  device='cuda:0'
  print('GPU')
else:
  device='cpu'
  print('CPU')

GPU


In [8]:
torch.cuda.empty_cache()

In [9]:
torch.cuda.get_device_name(0)

'GeForce RTX 2060'

In [10]:
torch.cuda.device_count()

1

In [18]:
model=model.to(device)

In [19]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [23]:
def evaluate(model,dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [21]:
training_loss=[]
validation_loss=[]

In [22]:
for epoch in tqdm(range(1,num_epochs+1)):
  model.train()
  total_training_loss=0
  progress_bar=tqdm(dataloader_train,
                    desc=f'Epoch : {epoch}',
                    leave=False,
                    disable=False)
  
  for batch in progress_bar:
    model.zero_grad()
    batch=tuple(item.to(device) for item in batch)

    inputs={
        'input_ids':batch[0],
        'attention_mask':batch[1],
        'labels':batch[2]
    }
    
    outputs=model(**inputs)
    loss=outputs[0]

    total_training_loss+=loss.item()
    loss.backward()

    #preform gradient clipping for safety
    #here all weights per layer will be of norm 1
    torch.nn.utils.clip_grad_norm(model.parameters(),1.0)

    optimizer.step()
    scheduler.step()

    progress_bar.set_postfix({'training_loss':f'{loss.item()/len(batch)}' })
  
  torch.save(model.state_dict(),f'BERT_hyperpartisan_model_epoch{epoch}.model')

  tqdm.write(f'\n Epoch {epoch}')

  loss_train_average=total_training_loss/len(dataloader_train)
  tqdm.write(f'Training Loss: {loss_train_average}')
  training_loss.append(total_training_loss/len(dataloader_train))
  
  val_loss,predictions,true_values=evaluate(dataloader_val)
  validation_loss.append(val_loss)
  val_f1=get_f1_score(predictions,true_values)
  tqdm.write(f'Validation_Loss: {val_loss} ')
  tqdm.write(f'Weighted F1_score : {val_f1}')

d:\python\lib\site-packages\ipykernel_launcher.py:27: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.



 Epoch 1
Training Loss: 0.20791213685557247
Validation_Loss: 0.13134285325929523 
Weighted F1_score : 0.951559258408763



 Epoch 2
Training Loss: 0.14583279300133387
Validation_Loss: 0.08702782385547955 
Weighted F1_score : 0.9708833315135416


KeyboardInterrupt: 

EVALUATE

In [13]:
loaded_model=BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)

In [14]:
status=loaded_model.to(device)

In [15]:
loaded_model.load_state_dict(torch.load('./BERT_hyperpartisan_model_epoch2.model'))

<All keys matched successfully>

In [16]:
dataloader_test

In [17]:
def evaluate_test(loaded_model,dataloader_test):

    loaded_model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_test):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                 }

        with torch.no_grad():        
            outputs = loaded_model(**inputs)
            
        logits = outputs[0]
    
        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    
    
    predictions = np.concatenate(predictions, axis=0)
    
    return  predictions


In [25]:
predictions=evaluate_test(loaded_model,dataloader_test)

In [28]:
actuals=np.argmax(predictions,axis=1)

In [29]:
np.unique(actuals,return_counts=True)

(array([0, 1], dtype=int64), array([109970,  40030], dtype=int64))

In [36]:
label_dict.items()

dict_items([('true', 0), ('false', 1)])

In [38]:
act_dict={'0':'true','1':'false'}

In [39]:
real_preds=[]
for ele in actuals:
    real_preds.append(act_dict[str(ele)])

In [41]:
from sklearn.metrics import accuracy_score,f1_score

In [42]:
accuracy_score(real_preds,df_test['hyperpartisan'].values)

0.5656133333333333